In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to c:\users\arleen\appdata\local\temp\pip-req-build-5seceac5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10816 sha256=f8421a13a1d968c343e127dd1b27695b95cd01e74b1744d2967a71d4742a0e44
  Stored in directory: C:\Users\Arleen\AppData\Local\Temp\pip-ephem-wheel-cache-jn4es55u\wheels\ef\1d\c6\f7e47f1aa1bc9d05c4120d94f90a79cf28603ef343b0dd43ff
Successfully built nvcc4jupyter


  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git 'C:\Users\Arleen\AppData\Local\Temp\pip-req-build-5seceac5'

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%writefile 4.txt

4
0. 2. 3. 4.
2. 6. 4. 8.
8. 10. 6. 2.
12. 6. 8. 16.

Writing 4.txt


In [4]:
%%writefile matrix.h

#ifndef MATRIX_H
#define MATRIX_H

typedef struct Matrix {
    int col;
    int row;
    double* buffer;
} Matrix;

Matrix createMatrix(int row, int col);
void freeMatrix(Matrix *mat);
Matrix readMatrixFromFile();
Matrix createIdentityMatrix(int size);
void printMatrix(Matrix matrix);
double* getColFromMatrix(Matrix m, size_t colNum);

#endif

Writing matrix.h


In [10]:
%%writefile matrix.c

#include <stdio.h>
#include <stdlib.h>
#include <stddef.h>
#include "matrix.h"

Matrix createMatrix(int row, int col) {
    Matrix matrix;
    matrix.col = col;
    matrix.row = row;

    matrix.buffer = (double *)malloc(col * row * sizeof(double));
    if (matrix.buffer == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        exit(1);
    }

    return matrix;
}

void freeMatrix(Matrix *matrix) {
    free(matrix->buffer);

    matrix->buffer = NULL;
    matrix->col = 0;
    matrix->row = 0;
}

Matrix readMatrixFromFile(){
    int size;

    // Read matrix size
    if (fscanf(stdin, "%d", &size) != 1) {
        printf("Error reading matrix size.\n");
        exit(1);
    }

    Matrix matrix = createMatrix(size, size);

    // Size validation
    if (matrix.col <= 0 || matrix.row <= 0 || matrix.col % 4 != 0 || matrix.row % 4 != 0) {
        printf("Invalid matrix size.\n");
        exit(1);
    }

    // Read matrix buffer
    for (int i = 0; i < matrix.row; i++) {
        for (int j = 0; j < matrix.col; j++) {
            if (fscanf(stdin, "%lf", &(matrix.buffer[i * matrix.col + j])) != 1) {
                printf("Error reading file.\n");
                exit(1);
            }
        }
    }
    return matrix;
}

Matrix createIdentityMatrix(int size){
    Matrix matrix = createMatrix(size, size);
    for (int i = 0; i < size; i++) {
        for (int j = 0; j < size; j++) {
            if (i == j) {
                matrix.buffer[i * size + j] = 1.0;
            } else {
                matrix.buffer[i * size + j] = 0.0;
            }
        }
    }
    return matrix;
}

void printMatrix(Matrix matrix){
    for (int i = 0; i < matrix.row; i++) {
        for (int j = 0; j < matrix.col; j++) {
            printf("%.6f ", matrix.buffer[i * matrix.col + j]);
        }
        printf("\n");
    }
}

double* getColFromMatrix(Matrix m, size_t colNum){
    double* col = (double *)malloc(m.row * sizeof(double));
    if (col == NULL) {
        fprintf(stderr, "Memory allocation failed\n");
        exit(1);
    }
    for (size_t i = 0; i < m.row; i++){
        col[i] = m.buffer[i * m.col + colNum];
    }
    return col;
}

Overwriting cuda.cu


In [6]:
%%writefile serial.c


#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <string.h>
#include "matrix.h"

/*Getter: row in a certain index*/
double* getRow (Matrix m, size_t rowNum){
    double* row = (double *)calloc(m.col, sizeof(double));
    memcpy(row, &(m.buffer[rowNum * m.row]), m.row * sizeof(double));
    return row;
}

/*Swap row with another row*/
void swapRow(Matrix* matrix, int row1, int row2){
    if(row1 < 0 || row1 >= matrix->row || row2 < 0 || row2 >= matrix->row){
        fprintf(stderr, "Invalid row indices\n");
        return;
    }

    int startIndexRow2 = row2 * matrix->col;
    int startIndexRow1 = row1 * matrix->col;

    double* temp_row = (double*)malloc(matrix->col * sizeof(double));
    if(temp_row == NULL){
        fprintf(stderr, "Memory allocation failed\n");
        return;
    }

    // Copy row1 to temp_row
    memcpy(temp_row, &(matrix->buffer[startIndexRow1]), matrix->col * sizeof(double));

    // Copy row2 to row1
    memcpy(&(matrix->buffer[startIndexRow1]), &(matrix->buffer[startIndexRow2]), matrix->col * sizeof(double));

    // Copy temp_row to row2
    memcpy(&(matrix->buffer[startIndexRow2]), temp_row, matrix->col * sizeof(double));

    free(temp_row);
}

int main(void) {
    Matrix inputMatrix;
    Matrix identityMatrix;
    int size;

    inputMatrix = readMatrixFromFile();
    identityMatrix = createIdentityMatrix(inputMatrix.col);
    size = inputMatrix.col;

    int blockSize = 256;
    int numBlocks = (size + blockSize - 1) / 256;

    bool invertible = true;

    clock_t start = clock();

    for (size_t i = 0; i < size; i++){
        /* Partial Pivoting */
        /* Swapping indivisible row */
        double* colBuffer = getColFromMatrix(inputMatrix, i);

        if (colBuffer[i] == 0.){
            // Swap rows
            // search for the nearest non-zero row
            for (size_t swapIdx = i+1; swapIdx < size; swapIdx++){
                if (colBuffer[swapIdx] != 0.){
                    {
                        swapRow(&inputMatrix, i, swapIdx);
                        swapRow(&identityMatrix, i, swapIdx);
                    }
                    break;
                } else if (swapIdx == size - 1){
                    {
                        invertible = false;
                        fprintf(stderr, "Matrix can not be inversed.\n");
                    }
                }
            }
        }
        // Ensure all threads have checked invertibility before proceeding
        if (!invertible) {
            exit(1);
        }
    }

    printf("Input matrix after pivoting:\n");
    printMatrix(inputMatrix);
    printf("Identity matrix after pivoting:\n");
    printMatrix(identityMatrix);

    /* Eliminating */
    /* Reducing to upper triangle matrix */
    for (size_t i = 0; i < size; i++){
        for (size_t j = 0; j < size; j++){
            if (i != j){
                double eliminateFactor = inputMatrix.buffer[j * size + i] / inputMatrix.buffer[i * size + i];
                for (size_t k = 0; k < size; k++){
                    inputMatrix.buffer[j * size + k] -= inputMatrix.buffer[i * size + k] * eliminateFactor;
                    identityMatrix.buffer[j * size + k] -= identityMatrix.buffer[i * size + k] * eliminateFactor;
                }
            }
        }
    }

    printf("Input matrix after eliminate:\n");
    printMatrix(inputMatrix);
    printf("Identity matrix after eliminate:\n");
    printMatrix(identityMatrix);

    /* Reducing to diagonal matrix */
    for (size_t i = 0; i < size; i++){
        /* Divide the row by the pivot factor */
        double pivotFactor = inputMatrix.buffer[i * size + i];

        for (int j = 0; j < size; j++){
            inputMatrix.buffer[i * size + j] /= pivotFactor;
            identityMatrix.buffer[i * size + j] /= pivotFactor;
        }
    }

    printf("Result:\n");
    printf("%d\n",size);
    printMatrix(identityMatrix);

    clock_t end = clock();
    double exectime = (double) (end - start) / CLOCKS_PER_SEC;
    printf("Time taken is %.6f\n", exectime);

    return 0;
}

Writing serial.c


In [7]:
!gcc serial.c matrix.c -o serial
!./serial < 4.txt

'.' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
%%writefile cuda.cu
#include <time.h>
#include <stdio.h>
#include <stdlib.h>
#include <stdbool.h>
#include <string.h>

extern "C"{
  #include "matrix.h"
};

__device__ void printBuffer(double* buf, int *size){
    for (int i = 0; i < *size; i++){
        for (int j = 0; j < *size; j++){
            printf("%.4f\t", buf[i*(*size) + j]);
        }
        printf("\n");
    }
}

__device__ double* getColFromMatrixBuffer(double* m, int *size, int colNum){
    double* col = (double *)malloc((*size) * sizeof(double));
    if (col == NULL) {
        printf("Memory allocation failed\n");
        return;
    }
    for (int i = 0; i < *size; i++){
        col[i] = m[i * (*size) + colNum];
    }
    return col;
}


/*Getter: row in a certain index*/
double* getRow (Matrix m, int rowNum){
    double* row = (double *)calloc(m.col, sizeof(double));
    memcpy(row, &(m.buffer[rowNum * m.row]), m.row * sizeof(double));
    return row;
}

/*Swap row with another row*/
__device__ void swapRow(double* matrix, int row1, int row2, int *size){
    int startIndexRow2 = row2 * (*size);
    int startIndexRow1 = row1 * (*size);

    double* temp_row = (double*)malloc((*size) * sizeof(double));
    if(temp_row == NULL){
        printf("Memory allocation failed\n");
        return;
    }

    // Copy row1 to temp_row
    memcpy(temp_row, &(matrix[startIndexRow1]), (*size) * sizeof(double));

    // Copy row2 to row1
    memcpy(&(matrix[startIndexRow1]), &(matrix[startIndexRow2]), (*size) * sizeof(double));

    // Copy temp_row to row2
    memcpy(&(matrix[startIndexRow2]), temp_row, (*size) * sizeof(double));

    free(temp_row);
}

__global__ void pivoting(double *inputBuffer, double *identityBuffer, int *size, bool *invertible){
    // printf("PIVOTT %d\n", *size);
    // __shared__ double* inputBufferShared;
    // __shared__ double* identityBufferShared;

    // Calculate thread ID
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    // printf("After tid...\n");

    // Load elements into shared memory
    // inputBufferShared[threadIdx.x] = inputBuffer[tid];
    // __syncthreads();
    // printf(
    //   "Block %d, Thread %d, inputBufferShared=[%.4f, %.4f, %.4f, %.4f]\n",
    //   blockIdx.x, threadIdx.x,
    //   inputBufferShared[0], inputBufferShared[1], inputBufferShared[2], inputBufferShared[3]
    // );


    // identityBufferShared[threadIdx.x] = identityBuffer[tid];
    // __syncthreads();

    // printf("Before syncthread...\n");


    for (int i = 0; i < *size; i++){
        printf("pivot bng\n");
        /* Partial Pivoting */
        /* Swapping indivisible row */
        double* colBuffer = getColFromMatrixBuffer(inputBuffer, size, i);

        for (int q=0; q<*size; q++){
          printf("%.4f\t", colBuffer[q]);
        }
        printf("\n");

        if (colBuffer[i] == 0.){
            // Swap rows
            // search for the nearest non-zero row
            for (int swapIdx = i+1; swapIdx < *size; swapIdx++){
                if (colBuffer[swapIdx] != 0.){
                    {
                        swapRow(inputBuffer, i, swapIdx, size);
                        __syncthreads();
                        swapRow(identityBuffer, i, swapIdx, size);
                        __syncthreads();

                        printf("Input matrix after swap row:\n");
                        printBuffer(inputBuffer, size);
                        printf("Identity matrix after swap row:\n");
                        printBuffer(identityBuffer, size);
                    }
                    break;
                } else if (swapIdx == *size - 1){
                    {
                        *invertible = false;
                        printf("Matrix can not be inversed.\n");
                    }
                }
            }
        }
    }

    // if (threadIdx.x == 0) {
    //   inputBuffer[blockIdx.x] = inputBufferShared[0];
    //   identityBuffer[blockIdx.x] = identityBufferShared[0];
    // }
}

__global__ void eliminate(double *inputBuffer, double *identityBuffer, int size){
    int row = blockIdx.x * blockDim.x + threadIdx.x;
    if (row < size){
        double diagonal_element = inputBuffer[row * size + row];
        printf("Row: %d, diagonal: %.4f\n", row, diagonal_element);
        for (int j = 0; j < size; j++){
            if (j != row) {
                double eliminateFactor = inputBuffer[j * size + row] / diagonal_element;
                printf("Eliminate: %.4f\n", eliminateFactor);
                for (int k = 0; k < size; k++) {
                    inputBuffer[j * size + k] -= inputBuffer[row * size + k] * eliminateFactor;
                    identityBuffer[j * size + k] -= identityBuffer[row * size + k] * eliminateFactor;
                }
                __syncthreads();
            }
        }
        __syncthreads();
    }
}


int main(void) {
    Matrix inputMatrix;
    Matrix identityMatrix;
    int size;

    inputMatrix = readMatrixFromFile();
    identityMatrix = createIdentityMatrix(inputMatrix.col);
    size = inputMatrix.col;

    dim3 block(256,1,1);
    dim3 grid((size + block.x - 1) / block.x, 1, 1);

    bool invertible = true;

    // GPU allocation memory
    double *d_inputMatrix, *d_identityMatrix;
    bool *d_invertible;
    int *d_size;
    cudaMalloc((void **)&d_inputMatrix, size*size*sizeof(double));
    cudaMalloc((void **)&d_identityMatrix, size*size*sizeof(double));
    cudaMalloc((void **)&d_invertible, sizeof(bool));
    cudaMalloc((void **)&d_size, sizeof(int));

    cudaMemcpy(d_inputMatrix, inputMatrix.buffer, size * size * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_identityMatrix, identityMatrix.buffer, size * size * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_invertible, &invertible, sizeof(bool), cudaMemcpyHostToDevice);
    cudaMemcpy(d_size, &size, sizeof(int), cudaMemcpyHostToDevice);

    clock_t start = clock();

    /* Partial Pivoting */
    pivoting<<<grid,block>>>(d_inputMatrix, d_identityMatrix, d_size, d_invertible);
    cudaDeviceSynchronize();

    // Ensure all threads have checked invertibility before proceeding
    if (!invertible) {
        return;
    }


    cudaMemcpy(inputMatrix.buffer, d_inputMatrix, size * size * sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(identityMatrix.buffer, d_identityMatrix, size * size * sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(&invertible, d_invertible, sizeof(bool), cudaMemcpyDeviceToHost);
    cudaMemcpy(&size, d_size, sizeof(int), cudaMemcpyDeviceToHost);


    printf("Input matrix after pivoting:\n");
    printMatrix(inputMatrix);
    printf("Identity matrix after pivoting:\n");
    printMatrix(identityMatrix);

    // GPU copy memory
    cudaMemcpy(d_inputMatrix, inputMatrix.buffer, size * size * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_identityMatrix, identityMatrix.buffer, size * size * sizeof(double), cudaMemcpyHostToDevice);

    /* Eliminating */
    /* Reducing to upper triangle matrix */
    // for (int i = 0; i < size; i++){
    //     for (int j = 0; j < size; j++){
    //         if (i != j){
    //             double eliminateFactor = inputMatrix.buffer[j * size + i] / inputMatrix.buffer[i * size + i];
    //             for (int k = 0; k < size; k++){
    //                 inputMatrix.buffer[j * size + k] -= inputMatrix.buffer[i * size + k] * eliminateFactor;
    //                 identityMatrix.buffer[j * size + k] -= identityMatrix.buffer[i * size + k] * eliminateFactor;
    //             }
    //         }
    //     }
    // }
    eliminate<<<grid,block>>>(d_inputMatrix, d_identityMatrix, size);
    cudaDeviceSynchronize();

    cudaMemcpy(inputMatrix.buffer, d_inputMatrix, size * size * sizeof(double), cudaMemcpyDeviceToHost);
    cudaMemcpy(identityMatrix.buffer, d_identityMatrix, size * size * sizeof(double), cudaMemcpyDeviceToHost);

    printf("Input matrix after eliminate:\n");
    printMatrix(inputMatrix);
    printf("Identity matrix after eliminate:\n");
    printMatrix(identityMatrix);

    /* Reducing to diagonal matrix */
    for (int i = 0; i < size; i++){
        /* Divide the row by the pivot factor */
        double pivotFactor = inputMatrix.buffer[i * size + i];

        for (int j = 0; j < size; j++){
            inputMatrix.buffer[i * size + j] /= pivotFactor;
            identityMatrix.buffer[i * size + j] /= pivotFactor;
        }
    }

    printf("%d\n",size);
    fprintf(stderr, "Result.\n");
    printMatrix(identityMatrix);

    clock_t end = clock();
    double exectime = (double) (end - start) / CLOCKS_PER_SEC;
    printf("Time taken is %.6f\n", exectime);

    cudaFree(d_inputMatrix);
    cudaFree(d_identityMatrix);
    cudaFree(d_invertible);

    return 0;
}

Overwriting cuda.cu


In [12]:
!nvcc cuda.cu matrix.c -o cuda
!./cuda < 4.txt

nvcc fatal   : Cannot find compiler 'cl.exe' in PATH


'.' is not recognized as an internal or external command,
operable program or batch file.
